---
## Vamos a crear una base de datos de alumnos 

---

In [7]:
# Zona de imports
import mysql.connector
import db_config as c  # Aca estan todos los datos necesarios para conectarse a la base de datos



---
#### Verificamos las bases de datos actuales

---

In [8]:
# Creamos una funcion para mostrar las bases de datos disponibles

def show_db():
    # Nos conectamos al host con nuestros datos
    mydb = mysql.connector.connect(
        host=c.config['host'],
        user=c.config['user'],
        password=c.config['password']
    )

    # Creamos un cursor
    mycursor = mydb.cursor()   

    # Armamos la query 
    query = "SHOW DATABASES"

    # Ejecutamos
    mycursor.execute(query)

    # Mostramos resultados de la consulta
    for x in mycursor:
        print(x)
    
    # Cerramos
    mycursor.close()
    mydb.close()

In [9]:
# Ejecuto la funcion

show_db()

('basedenoe',)
('holamundo',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


---
#### Creamos una base de datos llamada facultad

> En esta base de datos crearemos las tabla alumnos, docentes, asignaturas, carreras

---

In [10]:
def create_table(table_name):
    # Nos conectamos con nuestros datos
    mydb = mysql.connector.connect(
        host=c.config['host'],
        user=c.config['user'],
        password=c.config['password']
    )

    mycursor = mydb.cursor()

    # Creo la query y la ejecuto
    query = "CREATE DATABASE "+table_name
    mycursor.execute(query)

    # Cierro
    mycursor.close()
    mydb.close()
    


In [11]:
# Creamos la base de datos, invocando la funcion
create_table("facultad")

In [12]:
# Mostramos si se ha creado la base de datos
show_db()

('basedenoe',)
('facultad',)
('holamundo',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


---
### Conectarse a una Base de Datos 
Antes de crear una tabla dentro de una BD, debemos conectarnos a ella (indicar cual de todas las BD vamos a utilizar)

---

In [13]:
def connect_to_db(db_name):

    mydb = mysql.connector.connect( 
        host=c.config['host'],
        user=c.config['user'],
        password=c.config['password'],
        database=db_name
    )

    return mydb

---
#### Creamos una tabla de alumnos

---

In [14]:
# Podemos definir una funcion para crear tablas en forma generica
def create_table(db_name, table_name, col_dict):
    """
    col_dict = {
        col1: {col_name: name, col_type: type},
        col2: {col_name: name, col_type: type},
        col3: {col_name: name, col_type: type},
        ...
        coln: {col_name: name, col_type: type},
    }
    """
    my_str = ""

    for k,v in col_dict.items():
        my_str += (v['col_name'] + " "+v['col_type']+",")

    query = "CREATE TABLE "+table_name+" (id INT AUTO_INCREMENT PRIMARY KEY, "+ my_str[:-1] + ")"
    
    
    mydb = connect_to_db(db_name)
    mycursor = mydb.cursor()
    mycursor.execute(query)
    mycursor.close()
    mydb.close()



In [15]:
# Creamos nuestra tabla de alumnos
columnas = ['nombre', 'apellido', 'fecha_nacimento', 'dni', 'email', 'carrera', 'promedio']
tipos = ["VARCHAR(100)", "VARCHAR(100)", "DATE", "VARCHAR(20)", "VARCHAR(50)", "VARCHAR(30)", "FLOAT"]

def create_dict(columnas, tipos):
    my_dict = {}

    paquetes = list(zip(columnas, tipos))


    for idx, paquete in enumerate(paquetes):
        my_dict["col"+str(idx+1)] = {"col_name":paquete[0], "col_type": paquete[1]}

    return my_dict

my_dict = create_dict(columnas, tipos)

In [16]:
# Creamos la tabla
create_table("facultad", "alumnos", my_dict)

In [17]:
# Creamos una funcion para mostrar la tabla creada (y las demas)
def show_tables_in_db(db_name):
    mydb = connect_to_db(db_name)

    mycursor = mydb.cursor()
    mycursor.execute("SHOW TABLES")

    for x in mycursor:
        print(x)

    mycursor.close()
    mydb.close()

In [18]:
# Invoco la funcion
show_tables_in_db("facultad")

('alumnos',)


---
#### Funciones para agregar alumnos de a 1 
---

In [19]:
## Funciones auxiliares

def format_column_names(column_names, values):
    col_str = ""

    n_col = len(column_names)
    for idx, col in enumerate(column_names):
        col_str += col
        if idx != n_col-1:
            col_str += ", "

    
    n_val = len(values)
    val_str = ""
    for i in range(n_val):
        val_str += "%s"
        if i!=n_val-1:
            val_str +=", "

    return col_str, val_str

def get_alumnos_col_names():
    return ['nombre', 'apellido', 'fecha_nacimento', 'dni', 'email', 'carrera', 'promedio']

In [20]:
def insert_1_into_table(db_name, table_name, column_names, values):

    mydb = connect_to_db(db_name)
    mycursor = mydb.cursor()

    col_str, val_str = format_column_names(column_names, values)

    sql = "INSERT INTO "+ table_name +" ("+col_str+") VALUES ("+val_str+")"
    val = values
    
    # print("sql --> ", sql)
    # print("val --> ", val)
    
    
    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

    mycursor.close()
    mydb.close()

In [21]:
# Inserto un/a alumno/a


col_names = get_alumnos_col_names()
values = ["Maria", "Luz", "2001-01-24", "00000112", "maria.luz@email.com", "Computacion", 7.35]

insert_1_into_table("facultad", "alumnos", col_names, values)

1 record inserted.


---
#### Funcion para agregar alumnos de a varios a la vez

---

In [22]:

def insert_several_into_table(db_name, table_name, column_names, values):
    
    mydb = connect_to_db(db_name)
    mycursor = mydb.cursor()
    
    col_str, val_str = format_column_names(column_names, values[0])

    sql = "INSERT INTO "+ table_name +" ("+col_str+") VALUES ("+val_str+")"
    # print("sql --> ", sql) 
    # print("values --> ", values)
    mycursor.executemany(sql, values)

    mydb.commit()

    print(mycursor.rowcount, "was inserted.")
    mycursor.close()
    mydb.close()

In [23]:
# Agrego varios

col_names = get_alumnos_col_names()
print(col_names)
values = [
    ["Juliana", "Pepe", "1999-04-18", "00000113", "juliana.pepe@email.com", "Alimentos", 6.28],
    ["Ramiro", "Roa", "2001-11-28", "00000114", "ramiro.roa@email.com", "Mecanica", 5.45],
    ["Gonzalo", "Yule", "2002-06-14", "00000115", "gonzalo.yule@email.com", "Electrica", 7.67]
    ]
insert_several_into_table("facultad", "alumnos", col_names, values)

['nombre', 'apellido', 'fecha_nacimento', 'dni', 'email', 'carrera', 'promedio']
3 was inserted.


---
#### La tabla completa --> SELECT statement

---

In [24]:
def show_table(db_name, table_name):
    mydb = connect_to_db(db_name)
    mycursor = mydb.cursor()
    
    sql = "SELECT * FROM "+table_name
    mycursor.execute(sql)

    myresult = mycursor.fetchall()

    for x in myresult:
        print(x)

    mycursor.close()
    mydb.close()

In [25]:
show_table("facultad", "alumnos")

(1, 'Maria', 'Luz', datetime.date(2001, 1, 24), '00000112', 'maria.luz@email.com', 'Computacion', 7.35)
(2, 'Juliana', 'Pepe', datetime.date(1999, 4, 18), '00000113', 'juliana.pepe@email.com', 'Alimentos', 6.28)
(3, 'Ramiro', 'Roa', datetime.date(2001, 11, 28), '00000114', 'ramiro.roa@email.com', 'Mecanica', 5.45)
(4, 'Gonzalo', 'Yule', datetime.date(2002, 6, 14), '00000115', 'gonzalo.yule@email.com', 'Electrica', 7.67)


---
#### Agrego varios alumnos mas para poder practicar filtrado

---

In [26]:
col_names = get_alumnos_col_names()
print(col_names)

values = [
    ["Rosario", "Gomez", "2001-07-21", "00000116", "rosario.gomez@email.com", "Alimentos", 8.38],
    ["Lucas", "Note", "1998-01-16", "00000117", "lucas.note@email.com", "Electronica", 8.75],
    ["Luisana", "Cel", "2000-03-04", "00000118", "luisana.cel@email.com", "Mecanica", 7.07],
    ["Camilo", "Septimo", "1997-08-11", "00000119", "camilo.septimo@email.com", "Electronica", 7.77],
    ["Marita", "Chizo", "1996-05-23", "00000120", "marita.chizo@email.com", "Computacion", 8.75]    
    ]
insert_several_into_table("facultad", "alumnos", col_names, values)
show_table("facultad", "alumnos")

['nombre', 'apellido', 'fecha_nacimento', 'dni', 'email', 'carrera', 'promedio']
5 was inserted.
(1, 'Maria', 'Luz', datetime.date(2001, 1, 24), '00000112', 'maria.luz@email.com', 'Computacion', 7.35)
(2, 'Juliana', 'Pepe', datetime.date(1999, 4, 18), '00000113', 'juliana.pepe@email.com', 'Alimentos', 6.28)
(3, 'Ramiro', 'Roa', datetime.date(2001, 11, 28), '00000114', 'ramiro.roa@email.com', 'Mecanica', 5.45)
(4, 'Gonzalo', 'Yule', datetime.date(2002, 6, 14), '00000115', 'gonzalo.yule@email.com', 'Electrica', 7.67)
(5, 'Rosario', 'Gomez', datetime.date(2001, 7, 21), '00000116', 'rosario.gomez@email.com', 'Alimentos', 8.38)
(6, 'Lucas', 'Note', datetime.date(1998, 1, 16), '00000117', 'lucas.note@email.com', 'Electronica', 8.75)
(7, 'Luisana', 'Cel', datetime.date(2000, 3, 4), '00000118', 'luisana.cel@email.com', 'Mecanica', 7.07)
(8, 'Camilo', 'Septimo', datetime.date(1997, 8, 11), '00000119', 'camilo.septimo@email.com', 'Electronica', 7.77)
(9, 'Marita', 'Chizo', datetime.date(1996, 5,

---
#### DROP TABLE (if exist)

##### **CUIDADO!!** Eliminar una tabla es un proceso irreversible.

---

In [27]:
def drop_table_if_exist(db_name, table_name):
    mydb = connect_to_db(db_name)
    mycursor = mydb.cursor()

    sql = "DROP TABLE IF EXISTS " + table_name

    
    mycursor.execute(sql)
    
    mycursor.close()
    mydb.close()

In [28]:
drop_table_if_exist("facultad", "alumnos")

---
#### Drop database

#### **Cuidado!** Eliminar una base de datos es un proceso irreversible

---

In [29]:
def drop_database_if_exist(db_name):
    mydb = connect_to_db(db_name)
    mycursor = mydb.cursor()

    sql = "DROP DATABASE IF EXISTS " + db_name

    
    mycursor.execute(sql)
    
    mycursor.close()
    mydb.close()

In [30]:
drop_database_if_exist("facultad")